# 3 More Namespace Operations 

### 3.1 `locals()` and `globals()`

Name binding operations covered so far:

  - *name* `=` (assignment)
  - `del` *name* (unbinds the name)
  - `def` *name* function definition (including lambdas)
  - `def name(`*names*`):` (function execution)
  - *name*`.`*attribute_name* `=`, `__setattr__`, `__delattr__`
  - `global`, `nonlocal` (changes scope rules)
  - `except Exception as` *name*:

In [ ]:
locals()

In [ ]:
len(locals())

  In the REPL these are the same:

In [ ]:
locals() == globals()

In [ ]:
x = 0

In [ ]:
x

  The following code is not recommended.

In [ ]:
locals()['x']

In [ ]:
locals()['x'] = 1

In [ ]:
locals()['x']

In [ ]:
x

  If you're tempted to use it, try this code which due to "fast
locals" doesn't do what you might expect:

In [ ]:
def f():
    locals()['x'] = 5
    print(x)
f()

### 3.2 The `import` Statement

In [ ]:
def _dir(obj='__secret', _CLUTTER=dir()):
    """
    A version of dir that excludes clutter and private names.
    """
    if obj == '__secret':
        names = globals().keys()
    else:
        names = dir(obj)
    return [n for n in names if n not in _CLUTTER and not n.startswith('_')]

In [ ]:
_dir()

In [ ]:
import csv
_dir()

In [ ]:
csv

In [ ]:
_dir(csv)

In [ ]:
csv.reader

In [ ]:
csv.writer

In [ ]:
csv.spam

In [ ]:
csv.spam = 'Python is dangerous'
csv.spam

In [ ]:
csv.reader = csv.writer
csv.reader

In [ ]:
from csv import reader as csv_reader
_dir()

In [ ]:
csv.reader is csv_reader

In [ ]:
csv

In [ ]:
csv.reader

In [ ]:
del csv
import csv as csv_module
_dir()

In [ ]:
csv_module.reader is csv_reader

In [ ]:
csv_module.reader

In [ ]:
math

In [ ]:
math + 3

In [ ]:
del math

In [ ]:
print(math)

  Will the next statement give a `NameError` like the previous statement?  Why not?

In [ ]:
import math

In [ ]:
math

In [ ]:
del math

  What if we don't know the name of the module until run-time?

In [ ]:
import importlib

In [ ]:
importlib.import_module('math')

In [ ]:
math.pi

In [ ]:
math_module = importlib.import_module('math')

In [ ]:
math.pi

In [ ]:
math_module.pi

In [ ]:
module_name = 'math'

In [ ]:
import module_name

In [ ]:
import 'math'

In [ ]:
import math

### 3.3 Exercises: The `import` Statement

  Explore reloading a module.  This is rarely needed and usually only when exploring.

  Several statements below will throw errors - try to figure out which ones before you run them.

In [ ]:
import csv

In [ ]:
import importlib

In [ ]:
importlib.reload?

In [ ]:
del csv

In [ ]:
importlib.reload(csv)

In [ ]:
importlib.reload('csv')

In [ ]:
import csv

In [ ]:
importlib.reload('csv')

In [ ]:
importlib.reload(csv)

### 3.4 Augmented Assignment Statements

Bind two names to the `str` object `'abc'`, then from it create `'abcd'`
and rebind (reassign) one of the names:

In [ ]:
string_1 = string_2 = 'abc'
string_1 is string_2

In [ ]:
string_2 = string_2 + 'd'
string_1 is string_2, string_1, string_2

  This reassigns the second name so it is bound to a new
object.  This works similarly if we start with two names for one
`list` object and then reassign one of the names.

In [ ]:
list_1 = list_2 = ['a', 'b', 'c']
list_1 is list_2

In [ ]:
list_2 = list_2 + ['d']
list_1 is list_2, list_1, list_2

  If for the `str` objects we instead use an *augmented assignment
statement*, specifically *in-place add* `+=`, we get the same
behaviour as earlier.

In [ ]:
string_1 = string_2 = 'abc'

In [ ]:
string_2 += 'd'
string_1 is string_2, string_1, string_2

  However, for the `list` objects the behaviour changes.

In [ ]:
list_1 = list_2 = ['a', 'b', 'c']

In [ ]:
list_2 += ['d']
list_1 is list_2, list_1, list_2

  The `+=` in `foo += 1` is not just syntactic sugar for `foo = foo +
1`.  The `+=` and other augmented assignment statements have their
own bytecodes and methods.

  Notice BINARY_ADD vs. INPLACE_ADD.  The run-time types of the
objects to which `name_1` and `name_2` are bound are irrelevant to the
bytecode that gets produced.

In [ ]:
import codeop, dis

In [ ]:
dis.dis(codeop.compile_command("name_1 = name_1 + name_2"))

In [ ]:
dis.dis(codeop.compile_command("name_1 += name_2"))

In [ ]:
list_2 = ['a', 'b', 'c']

In [ ]:
list_2

  Notice that `__iadd__` returns a value

In [ ]:
list_2.__iadd__(['d'])

  and it also changes the list

In [ ]:
list_2

In [ ]:
string_2.__iadd__('4')


So what happens when `INPLACE_ADD` operates on the `str` object?

If `INPLACE_ADD` doesn't find `__iadd__` it instead calls `__add__` and
reassigns `string_2`, i.e. it falls back to `__add__`.

https://docs.python.org/3/reference/datamodel.html#object.__iadd__:

> These methods are called to implement the augmented arithmetic
> assignments (+=, etc.). These methods should attempt to do the
> operation in-place (modifying self) and return the result (which
> could be, but does not have to be, self). If a specific method is
> not defined, the augmented assignment falls back to the normal
> methods.


  Here's similar behaviour with a tuple:

In [ ]:
tuple_1 = (7,)
tuple_1

In [ ]:
tuple_1[0].__iadd__(1)

In [ ]:
tuple_1[0] += 1

In [ ]:
tuple_1[0] = tuple_1[0] + 1

In [ ]:
tuple_1

  Here's surprising behaviour with a tuple:

In [ ]:
tuple_2 = ([12, 13],)
tuple_2

In [ ]:
tuple_2[0] += [14]

  What value do we expect `tuple_2` to have?

In [ ]:
tuple_2

  Let's simulate the steps to see why this behaviour makes sense.

In [ ]:
list_1 = [12, 13]

In [ ]:
tuple_2 = (list_1,)

In [ ]:
tuple_2

In [ ]:
temp = list_1.__iadd__([14])

In [ ]:
temp

In [ ]:
temp == list_1

In [ ]:
temp is list_1

In [ ]:
tuple_2

In [ ]:
tuple_2[0] = temp

  For later study:

In [ ]:
dis.dis(codeop.compile_command("tuple_2 = ([12, 13],); tuple_2[0] += [14]"))

In [ ]:
dis.dis(codeop.compile_command("tuple_2 = ([12, 13],); temp = tuple_2[0].__iadd__([14]); tuple_2[0] = temp"))

  For a similar explanation see 
https://docs.python.org/3/faq/programming.html#faq-augmented-assignment-tuple-error

### 3.5 Function Arguments are Passed by Name Binding

  Can functions modify the arguments passed to them?

  When a caller passes an argument to a function, the function starts
  execution with a local name, the parameter from its signature, bound
  to the argument object passed in.

In [ ]:
def function_1(string_2):
    print('A -->', string_2)
    string_2 += ' blue'
    print('B -->', string_2)

In [ ]:
string_1 = 'red'
string_1

In [ ]:
function_1(string_1)

In [ ]:
string_1

  To see more clearly why `string_1` is still a name bound to `'red'`, consider
this version which is functionally equivalent but has two changes
highlighted in the comments:

In [ ]:
def function_2(string_2):
    print('A -->', string_2)
    string_2 = string_2 + ' blue'  # Changed from +=
    print('B -->', string_2)

In [ ]:
function_2('red')  # Changed from string_1 to 'red'

In [ ]:
'red'

  In both cases the name `string_2` at the beginning of `function_1` and
`function_2` was a name that was bound to the `str` object `'red'`,
and in both the function-local name `string_2` was re-bound to
the new `str` object `'red blue'`.

  Let's try this with a `list`.

In [ ]:
def function_3(list_2):
    print('A -->', list_2)
    list_2 += ['blue']  # += with lists is shorthand for list.extend()
    print('B -->', list_2)

In [ ]:
list_1 = ['red']
list_1

In [ ]:
function_3(list_1)

In [ ]:
list_1

  In both cases parameter names are bound to arguments, and whether or
not the function can or does change the object passed in depends on
the object, not how it's passed to the function.